### Step1 Generate test cases

First, we need to import necessary tools and load code segment database. 

In [1]:
import random
import ast
import os
os.chdir("../src/Generate")

from basic_operation.dict_operation import get_rest_args
from DBOperation.dboperation_sqlite import DataBaseHandle
from combine_fragment import CodeFragmentGenerator, get_contained_cons
from cons_generator.cwvp import generate_if_cons_exist
from generate import Generate
from class_for_info.fragment_info import CodeFragmentInfo

generate = Generate("../config.json")
fragment = CodeFragmentGenerator(1, "CodeFragment_CW")
frag_db = DataBaseHandle("../../data/query/corpus-v3.db")
frag_list = frag_db.selectAll("select * from CodeFragment_CW")
print("Done.")

Done.


Then, we can select a code segment from corpus, try to combine other segments with it, and generate all necessary input values. 

In [2]:
frag = random.choice(frag_list)
available_variables = ast.literal_eval(frag[2])
needful_variables = ast.literal_eval(frag[3])
print("\033[1mcontent of code segment:\033[0m\n"+frag[1]+
      "\033[1mpre-conditions:\033[0m",needful_variables,"\n\033[1mpost-conditions:\033[0m",available_variables)

content of code segment:
let ket0 = (Complex(1.0, 0.0), Complex(0.0, 0.0));
let angle6059 = (PI() * 5.0) / 7.0;
H(register6059[0]);
Exp([PauliZ, PauliZ], angle6059, register6059);
H(register6059[0]);
Exp([PauliX, PauliI], -angle6059, register6059);

pre-conditions: {'register6059': 'Qubit[]'} 
post-conditions: {'angle6059': 'Double'}


In [4]:
# Parse constraints
bool_expr_list, func_ret_list, quanternion_list, needful_args, partial_reset_stmt = \
    get_contained_cons(frag[1], needful_variables)
# print("bool_expr_list:",bool_expr_list,"func_ret_list:",func_ret_list)
if len(bool_expr_list) > 0 or len(func_ret_list) > 0:
    print("\033[94mThere exist constraints!Start to generate inputs.\033[0m")
    valid_stmt, invalid_stmt = \
        generate_if_cons_exist(bool_expr_list, func_ret_list, quanternion_list, needful_args)
    print("==valid==\n"+valid_stmt+"\n==invalid==\n"+invalid_stmt)
else:
    valid_stmt, invalid_stmt = "//no cons\n", "//no cons\n"
# If generation failed
if valid_stmt is None:
    print("\033[91m!!!Generation failed!\033[0m")
# Generate values for other variables
needful_variables = get_rest_args(needful_variables, needful_args)
this_fragment_info = CodeFragmentInfo(frag[1], available_variables,
                                      needful_variables, frag[4], 
                                      frag[5], frag[6])
combined_fragment, combined_import = fragment.generate_a_code_frag(this_fragment_info)
if combined_fragment is None:
    print("\033[91m!!!Generation failed!\033[0m")
else:
    combined_fragment = combined_fragment.replace("\n\n", "\n").replace("//no cons\n//no cons\n", "//no cons\n")
    print("\033[1mvalid_stmt:\033[0m\n"+valid_stmt+combined_fragment+\
          "\033[1minvalid_stmt:\033[0m\n"+invalid_stmt+combined_fragment)

valid_stmt:
//no cons
use register6059 = Qubit[1];
// Modify initial state(s) of qubit(s). 
ApplyToEach(T, register6059);
// Modify end. 
let ket0 = (Complex(1.0, 0.0), Complex(0.0, 0.0));
let angle6059 = (PI() * 5.0) / 7.0;
H(register6059[0]);
Exp([PauliZ, PauliZ], angle6059, register6059);
H(register6059[0]);
Exp([PauliX, PauliI], -angle6059, register6059);
DumpMachine();
Message($"{angle6059}");
ResetAll(register6059);
invalid_stmt:
//no cons
use register6059 = Qubit[1];
// Modify initial state(s) of qubit(s). 
ApplyToEach(T, register6059);
// Modify end. 
let ket0 = (Complex(1.0, 0.0), Complex(0.0, 0.0));
let angle6059 = (PI() * 5.0) / 7.0;
H(register6059[0]);
Exp([PauliZ, PauliZ], angle6059, register6059);
H(register6059[0]);
Exp([PauliX, PauliI], -angle6059, register6059);
DumpMachine();
Message($"{angle6059}");
ResetAll(register6059);



Finally, we will combine all elements into a complete test case. If there are API constraints in the relevant code segments, Upbeat will generate two test cases (one for valid inputs and another for invalid inputs). Otherwise, Upbeat will generate a single test case. 

In [5]:
valid_testcase, invalid_testcase = "", ""
# Get self_defined_callables
defined_callables = ""
for item in fragment.self_defined_callables:
    if isinstance(item, str):
        if item not in defined_callables:
            defined_callables += item
    else:
        if item.content not in defined_callables:
            defined_callables += item.content
# print("===defined_callables:\n"+defined_callables+"\n")
# Process valid test case
valid_fragment = valid_stmt+combined_fragment+partial_reset_stmt
valid_import = combined_import
valid_testcase = generate.assemble_testcase(valid_fragment, valid_import, [defined_callables])
print("\033[1m==valid_testcase==\n\033[0m",valid_testcase)
# Process invalid test case
if invalid_stmt is None or invalid_stmt == valid_stmt:
    print("Same as below.")
else:
    invalid_fragment = invalid_stmt+combined_fragment+partial_reset_stmt
    invalid_import = combined_import
    invalid_testcase = generate.assemble_testcase(invalid_fragment, invalid_import, [defined_callables])
    print("\033[1m==invalid_testcase==\n\033[0m",invalid_testcase)

==valid_testcase==
 namespace NISLNameSpace {
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.Convert;
    open Microsoft.Quantum.Math;
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Oracles;
    open Microsoft.Quantum.Logical;
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Arrays;
    open Microsoft.Quantum.Arithmetic;


    
    @EntryPoint()
    operation main() : Unit {
        //no cons
        use register6059 = Qubit[1];
        // Modify initial state(s) of qubit(s). 
        ApplyToEach(T, register6059);
        // Modify end. 
        let ket0 = (Complex(1.0, 0.0), Complex(0.0, 0.0));
        let angle6059 = (PI() * 5.0) / 7.0;
        H(register6059[0]);
        Exp([PauliZ, PauliZ], angle6059, register6059);
        H(register6059[0]);
        Exp([PauliX, PauliI], -angle6059, register6059);
        DumpMachine();
        Message($"{angle6059}");
        ResetAll(register6059);
        
    }
}
Same as below.


### Step2   Execute test cases
The test oracles of Upbeat come from (1) language-level testing via constraints or (2) differential testing. In language-level testing, any results deviate from expected behaviors, crashed and timeouts will be served as anomalous. 

In [6]:
os.chdir("../Fuzzing")

from Fuzzing.lib.Harness import *
from Fuzzing.lib.post_processor import *
from Generate.basic_operation.file_operation import initParams

# You can also change `valid_testcase`` into `invalid_testcase` if they are different
testcase_content = valid_testcase

print("\033[1m==language-level testing==\033[0m")
susp_flag = False
# Record expected behavior
if "//wrong" in testcase_content or "//invalid" in testcase_content:
    flag = 0
elif "//correct" in testcase_content or "//valid" in testcase_content:
    flag = 1
else:
    flag = -1
# Execute test case
output = execute(0, testcase_content, ["dotnet", "run"], False)
print("\033[1moutput1:\n\033[0m"+output.stdout)
# Detect anomalous
if  ((flag == 1 and output.returnCode != 0) or 
    (flag == 0 and output.returnCode == 0) or 
    output.outputClass in ["timout", "crash"]):
    print("\033[91m!!!find wrong cons\033[0m")
    susp_flag = True
else:
    print("\033[92mnothing happened\033[0m")

==language-level testing==
cmd:dotnet run
output1:
[]
/root/UPBEAT/src/Fuzzing/qsharpPattern/Program.qs(21,25): warning CS0219: The variable 'ket0' is assigned but its value is never used [/root/UPBEAT/src/Fuzzing/qsharpPattern/qsharpPattern.csproj]
Unhandled exception. System.InvalidOperationException: Both input arrays for Exp (paulis, targets), must be of same size.
 ---> Microsoft.Quantum.Intrinsic.Exp on /root/UPBEAT/src/Fuzzing/qsharpPattern/D:\a\1\s\submodules\qsharp-runtime\src\Simulation\TargetDefinitions\Intrinsic\Exp.qs:line 0
   at NISLNameSpace.main on /root/UPBEAT/src/Fuzzing/qsharpPattern/Program.qs:line 0
nothing happened


In differential testing, Upbeat will detect any inconsistency, crash, or timeout. 

Note that the voting scheme has filtered some simple faulty behaviors. For example, ToffoliSimulator only supports parts of basic gates, it will throw a `NotImplementedException` exception if the test cases contain unsupported callables. Upbeat does not compare this exception with other results. 

In [7]:
# differential testing activates only if language-level testing detects no anomalies
if not susp_flag and output.returnCode not in [134, 137]:
    outputs = [output]
    command_list = [["dotnet", "run", "-s", "SparseSimulator"],
                    ["dotnet", "run", "-s", "ToffoliSimulator"]]
    # execute on SparseSimulator and ToffoliSimulator
    for i, cmd in enumerate(command_list, start=2):
        tmp_output = execute(0, output.testcaseContent, cmd, False)
        outputs.append(tmp_output)
        print("\033[1moutput"+str(i)+":\n\033[0m"+tmp_output.stdout)
    # voting scheme
    vote(outputs, output.testcaseContent)

cmd:dotnet run -s SparseSimulator
output2:
[]
/root/UPBEAT/src/Fuzzing/qsharpPattern/Program.qs(21,25): warning CS0219: The variable 'ket0' is assigned but its value is never used [/root/UPBEAT/src/Fuzzing/qsharpPattern/qsharpPattern.csproj]
Unhandled exception. System.InvalidOperationException: Both input arrays for Exp (paulis, targets), must be of same size.
 ---> Microsoft.Quantum.Intrinsic.Exp on /root/UPBEAT/src/Fuzzing/qsharpPattern/D:\a\1\s\submodules\qsharp-runtime\src\Simulation\TargetDefinitions\Intrinsic\Exp.qs:line 0
   at NISLNameSpace.main on /root/UPBEAT/src/Fuzzing/qsharpPattern/Program.qs:line 0
cmd:dotnet run -s ToffoliSimulator
output3:
/root/UPBEAT/src/Fuzzing/qsharpPattern/Program.qs(21,25): warning CS0219: The variable 'ket0' is assigned but its value is never used [/root/UPBEAT/src/Fuzzing/qsharpPattern/qsharpPattern.csproj]
Unhandled exception. System.NotImplementedException: The method or operation is not implemented.
 ---> Microsoft.Quantum.Intrinsic.T on /ro

### Step3 Filter anomalies

UPBEAT is capable of filtering the anomalies into three types: (1) bugs that have already been analyzed, (2) faulty that have already been analyzed, and (3) new anomalies awaiting verification. 

In [11]:
os.chdir("../")

from Fuzzing.history_bug_filter import filter_boundary, filter_differential

if os.path.exists("new_anomalies.txt"):
    print("situation 1")
    with open("bug.txt", "r") as f1:
        content1 = f1.read()
    print("\033[1mBugs that have already been analyzed:\033[0m\n"+content1)
    with open("faulty.txt", "r") as f2:
        content2 = f2.read()
    print("\033[1mFaulty that have already been analyzed:\033[0m\n"+content2)
    with open("new_anomalies.txt", "r") as f3:
        content3 = f3.read()
    print("\033[1mNew anomalies awaiting verification:\033[0m\n"+content3)    
else:
    print("situation 2")
    result_db = DataBaseHandle("../../data/result/UPBEAT.db")
    history_db = DataBaseHandle("../../data/query/history-bugs.db")
    filter_boundary(result_db, history_db)
    filter_differential(result_db)

situation 1
Bugs that have already been analyzed:
3('Ceiling', 'OverflowException')

Faulty that have already been analyzed:
7('//correct', '//no cons')

New anomalies awaiting verification:

